In [ ]:
import numpy as np

In [ ]:
a = np.random.randn(2, 3) # a.shape = (2, 3)
b = np.random.randn(2, 1) # b.shape = (2, 1)
c = a + b

In [ ]:
#print (a)
#print(b)
print(c.shape)

In [ ]:
a.b

In [ ]:
a*b

In [ ]:
a'*b

In [ ]:
a.T

In [ ]:
a.T.dot(b)

In [ ]:
np.sum(np.dot(a.T,b))

In [ ]:
b.shape

In [ ]:
from scipy import ndimage, misc
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2

In [ ]:
fig = plt.figure()
plt.gray()  # show the filtered result in grayscale
ax1 = fig.add_subplot(121)  # left side
ax2 = fig.add_subplot(122)  # right side
ascent = misc.ascent()
result = ndimage.median_filter(ascent, size=20)
ax1.imshow(ascent)
ax2.imshow(result)
plt.show()

In [ ]:
print(type(fig))

In [ ]:
img_path = r'F:\TUhh\Sem 5\Project\practice\train.jpg'

In [ ]:
def dullrazor(img, lowbound=15, showimgs=True, filterstruc=3, inpaintmat=3):
    #grayscale
    imgtmp1 = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    #applying a blackhat
    filterSize =(filterstruc, filterstruc)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, filterSize) 
    imgtmp2 = cv2.morphologyEx(imgtmp1, cv2.MORPH_BLACKHAT, kernel)

    #0=skin and 255=hair
    ret, mask = cv2.threshold(imgtmp2, lowbound, 255, cv2.THRESH_BINARY)
    
    #inpainting
    img_final = cv2.inpaint(img, mask, inpaintmat ,cv2.INPAINT_TELEA)
    
    if showimgs:
       # print("_____DULLRAZOR_____")
        #plt.imshow(imgtmp1, cmap="gray")
       # plt.show()
       # plt.imshow(imgtmp2, cmap='gray')
       # plt.show()
       # plt.imshow(mask, cmap='gray')
       # plt.show()
       # plt.imshow(img_final)
       # plt.show()
       # print("___________________")

    return img_final

In [ ]:
img= plt.imread(img_path)
#plt.imshow(img)# to show image
plt.imshow(img)
plt.show()
newimg=dullrazor(img)
#img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
#median = cv2.medianBlur(img,5)

#compare= np.concatenate((img,median),axis=1)
#plt.imshow(compare)


In [ ]:
nim2=dullrazor(newimg)
nim3=dullrazor(nim2)
nim4=dullrazor(nim3)
nim5=dullrazor(nim4)
nim6=dullrazor(nim5)
plt.imshow(nim6)
plt.show()

In [ ]:
#type(img)
img.shape

In [ ]:
image = cv2.imread(r'F:\TUhh\Sem 5\Project\practice\train2.jpg')
image_resize = cv2.resize(image,(500,500))

grayScale = cv2.cvtColor(image_resize, cv2.COLOR_RGB2GRAY)

kernel = cv2.getStructuringElement(1,(17,17))
blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)

ret,threshold = cv2.threshold(blackhat,10,255,cv2.THRESH_BINARY)
final_image = cv2.inpaint(image_resize,threshold,1,cv2.INPAINT_TELEA)

grayS = cv2.cvtColor(final_image, cv2.COLOR_RGB2GRAY)

ret,thr = cv2.threshold(grayScale, 0, 255, cv2.THRESH_OTSU)
output_im = cv2.morphologyEx(thr, cv2.MORPH_CLOSE, kernel)
cv2.imshow("Display window", final_image)

cv2.waitKey(0)

# Flow Guided Hair Removal

In [9]:
import cv2
import numpy as np
import copy
import matplotlib.pyplot as plt

In [26]:
M_PI = 3.14159265358979323846
KERNEL = 5
COLOUR_OR_GRAY = 0
input_img = r'F:\TUhh\Sem 5\Project\practice\train2.jpg'
SIZE = (700,500,3)

In [27]:
flowField = np.zeros(SIZE, dtype=np.float32)
refinedETF = np.zeros(SIZE, dtype=np.float32)
gradientMag = np.zeros(SIZE, dtype=np.float32)
flowField.shape

(700, 500, 3)

# ETF

In [28]:
def initial_ETF(input_img, size):
    global flowField
    global refinedETF
    global gradientMag

    src_load = cv2.imread(input_img, COLOUR_OR_GRAY)
    src = cv2.resize(src_load,(700,500))
    src_n = np.zeros(size, dtype=np.float32)
    src_n = cv2.normalize(src.astype('float32'), None, 0.0, 1.0, cv2.NORM_MINMAX)

    # Generate grad_x and grad_y
    grad_x = []
    grad_y = []
    grad_x = cv2.Sobel(src_n, cv2.CV_32FC1, 1, 0, ksize=5)
    grad_y = cv2.Sobel(src_n, cv2.CV_32FC1, 0, 1, ksize=5)
    print(grad_x.shape,grad_y.shape)

    # Compute gradient
    gradientMag = cv2.sqrt(grad_x * 2.0 + grad_y * 2.0)
    gradientMag = cv2.normalize(gradientMag.astype('float32'), None, 0.0, 1.0, cv2.NORM_MINMAX)
    h, w = src.shape[0], src.shape[1]
    for i in range(h):
        for j in range(w):
            u = grad_x[i][j]
            v = grad_y[i][j]
            n = np.array([v, u, 0.0])
            cv2.normalize(np.array([v, u, 0.0]).astype('float32'), n)
            flowField[i] = n
    rotateFlow(flowField, flowField, 90.0)

In [29]:
def rotateFlow(src, dst, theta):
    theta = theta / 180.0 * M_PI;
    h, w = src.shape[0], src.shape[1]
    for i in range(h):
        for j in range(w):
            v = src[i][j]
            rx = v[0] * np.cos(theta) - v[1] * np.sin(theta)
            ry = v[1] * np.cos(theta) + v[0] * np.sin(theta)
            flowField[i][j] = [rx, ry, 0.0]

In [30]:
def refine_ETF(kernel):
    global flowField
    global refinedETF
    global gradientMag
    h_f, w_f = flowField.shape[0], flowField.shape[1]
    for r in range(h_f):
        for c in range(w_f):
            computeNewVector(c, r, kernel)
    flowField = copy.deepcopy(refinedETF)


In [31]:
# Paper's Eq(1)
def computeNewVector(x, y, kernel):
    global flowField
    global refinedETF
    global gradientMag
    t_cur_x = flowField[y][x]
    #print("T_cur",t_cur_x," x:",x," y:",y)
    t_new = (0, 0, 0)
    h_r, w_r = refinedETF.shape[0], refinedETF.shape[1]
    for r in range(y - kernel, y + kernel + 1):
        for c in range(x - kernel, x + kernel + 1):
            if (r < 0 or r >= h_r or c < 0 or c >= w_r):
                continue
            t_cur_y = flowField[r][c]
            a = np.array([x, y])
            b = np.array([c, r])
            phi = computePhi(t_cur_x, t_cur_y);
            w_s = computeWs(a, b, kernel);
            w_m = computeWm(gradientMag[y], gradientMag[r])
            w_d = computeWd(t_cur_x, t_cur_y)
            #print(w_m)
            #print(t_cur_y)
            print("tcury:",t_cur_y.shape,"w_m:",w_m.shape,"w_d:",w_d.shape)
            t_new += phi * w_s * t_cur_y.dot(w_m) * w_d
    n = t_new
    cv2.normalize(t_new, n)
    refinedETF[y][x] = n

In [32]:
# Paper's Eq(5)
def computePhi(x, y):
    if np.dot(x, y) > 0:
        return 1
    else:
        return -1


# Paper's Eq(2)
def computeWs(x, y, r):
    if np.linalg.norm(x - y) < r:
        return 1
    else:
        return 0


# Paper's Eq(3)
def computeWm(gradmag_x, gradmag_y):
    wm = (1 + np.tanh(gradmag_y - gradmag_x)) / 2
    return wm


# Paper's Eq(4)
def computeWd(x, y):
    return abs(x.dot(y))

In [33]:
# plot arrowline and save image.
def draw_arrowline(count, KERNEL):
    global flowField
    dis = cv2.imread(input_img, COLOUR_OR_GRAY)
    resolution = 10;
    h, w = dis.shape[0], dis.shape[1]
    for i in range(0, h, resolution):
        for j in range(0, w, resolution):
            v = flowField[i][j]
            p = (j, i)
            p2 = (int(j + v[1] * 5), int(i + v[0] * 5))
            dis = cv2.arrowedLine(dis, p, p2, (255, 0, 0), 1, 8, 0, 0.3)
    cv2.imwrite('etf_kernel' + str(KERNEL) + '_' + str(count) + '.png', dis)
    np.save('np_etf_kernel' + str(KERNEL) + '_' + str(count) + '.npy', flowField)
    cv2.imshow(dis)



initial_ETF(input_img, SIZE)
for i in range(10):
    refine_ETF(KERNEL)
    draw_arrowline(i, KERNEL)

(500, 700) (500, 700)
[0.5 nan nan nan 0.5 0.5 nan nan nan nan nan nan nan nan nan nan 0.5 0.5
 0.5 0.5 nan 0.5 0.5 0.5 0.5 0.5 nan nan nan nan nan nan 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 nan nan nan nan nan 0.5 0.5 nan nan nan nan nan nan nan
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 nan nan nan nan nan nan nan 0.5 0.5 0.5
 0.5 0.5 nan nan nan nan 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 nan nan nan nan nan nan nan nan nan 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 nan nan nan nan nan nan 0.5 0.5 0.5 nan nan nan nan nan 0.5 0.5 0.5
 0.5 nan nan nan nan nan 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 nan nan nan 0.5
 0.5 nan nan nan nan nan nan nan 0.5 0.5 0.5 nan nan nan nan nan nan nan
 nan 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 nan nan nan nan 0.5 0.5 0.5
 0.5 0.5 nan nan nan 0.5 nan nan nan nan nan 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 nan nan nan nan nan 0.5 0.5 0.5 0.5 0.5 0.5 nan nan nan nan nan nan 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 nan nan nan nan nan nan nan nan nan nan
 nan 0.5 0.5 0.5 nan 0.5 0.5 

ValueError: shapes (3,) and (700,) not aligned: 3 (dim 0) != 700 (dim 0)

In [ ]:
print()